In [ ]:
import os
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder


# Fungsi memuat data
def load_data(data_path, image_size=(28, 28)):
    X, y = [], []
    for label in os.listdir(data_path):
        label_path = os.path.join(data_path, label)
        if os.path.isdir(label_path) and not label.startswith('.'):  # Hindari folder tersembunyi
            for file_name in os.listdir(label_path):
                image_path = os.path.join(label_path, file_name)
                if file_name.endswith(('.png', '.jpg', '.jpeg')):  # Hanya file gambar
                    try:
                        img = Image.open(image_path).convert('L')  # Konversi ke grayscale
                        img = img.resize(image_size)  # Ubah ukuran
                        img = np.array(img) / 255.0  # Normalisasi
                        X.append(img)
                        y.append(label)
                    except Exception as e:
                        print(f"Error loading image {image_path}: {e}")
    return np.array(X), np.array(y)


# Set jalur dataset
dataset_folder = "./dataset"  # Ganti dengan jalur folder dataset Anda
train_path = os.path.join(dataset_folder, "train")
validation_path = os.path.join(dataset_folder, "validation")
test_path = os.path.join(dataset_folder, "test")

# Muat data train dan validation
X_train, y_train = load_data(train_path)
X_validation, y_validation = load_data(validation_path)

# Preprocessing
X_train = X_train.reshape(X_train.shape[0], 28, 28, 1)
X_validation = X_validation.reshape(X_validation.shape[0], 28, 28, 1)

encoder = LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_validation = encoder.transform(y_validation)

# Konversi label ke one-hot encoding
num_classes = len(np.unique(y_train))
y_train = to_categorical(y_train, num_classes=num_classes)
y_validation = to_categorical(y_validation, num_classes=num_classes)

# Split dataset train untuk testing internal
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Model JST
model = Sequential([
    Flatten(input_shape=(28, 28, 1)),
    Dense(128, activation='relu'),
    Dense(num_classes, activation='softmax')
])
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Training
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_validation, y_validation))

# Evaluasi
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")


# Fungsi prediksi
def predict_letter(folder, file_name):
    img_path = os.path.join(test_path, folder, file_name)
    img = Image.open(img_path).convert('L')
    img = img.resize((28, 28))
    img = np.array(img) / 255.0
    img = img.reshape(1, 28, 28, 1)
    prediction = model.predict(img)
    predicted_label = encoder.inverse_transform([np.argmax(prediction)])
    return predicted_label[0]


# Menanyakan input pengguna untuk folder dan nama file
folder = input("Masukkan nama folder (A-Z) di dalam folder 'test': ").upper()
while folder not in os.listdir(test_path):
    print("Folder tidak ditemukan. Coba lagi.")
    folder = input("Masukkan nama folder (A-Z) di dalam folder 'test': ").upper()

# Menampilkan daftar file gambar di dalam folder yang dipilih
folder_path = os.path.join(test_path, folder)
images = [f for f in os.listdir(folder_path) if f.endswith(('.png', '.jpg', '.jpeg'))]
print("Daftar gambar di folder", folder, ":")
for idx, image in enumerate(images):
    print(f"{idx + 1}. {image}")

# Menanyakan nama file gambar
file_index = int(input("Pilih nomor gambar untuk diprediksi: ")) - 1
file_name = images[file_index]

# Menampilkan gambar yang dipilih
img_path = os.path.join(folder_path, file_name)
img = Image.open(img_path)
plt.imshow(img, cmap='gray')
plt.axis('off')  # Menyembunyikan axis

# Menambahkan teks prediksi huruf di bawah gambar
predicted_letter = predict_letter(folder, file_name)

# Menampilkan prediksi di bawah gambar
plt.text(0.5, -0.1, f"Predicted Letter: {predicted_letter}", ha='center', va='center', fontsize=20, fontweight='bold', transform=plt.gca().transAxes)
plt.show()
